In [1]:
import pandas as pd
import numpy as np

import nltk
import gensim
from textaugment import Word2vec, Wordnet, EDA
from nltk.corpus import wordnet

import os 
os.getcwd()

'C:\\Users\\thanisb\\Documents\\Competition\\Zindi\\Tech4MentalHealth\\Notebook'

# Reading file

In [58]:
train_DF = pd.read_csv('../data/train_corrected_3.csv')
train_DF

,ID,text,label
0,SUAVK39Z,i feel that it was better i die am happy,Depression
1,9JDAGUV3,why do i get hallucinations ?,Drugs
2,419WR1LQ,i am stressed due to lack of financial support...,Depression
3,6UY7DX6Q,why is life important ?,Suicide
4,FYC0FTFB,how could i be helped to go through the depres...,Depression
...,...,...,...
611,BOHSNXCN,what should i do to stop alcoholism ?,Alcohol
612,GVDXRQPY,how to become my old self again,Suicide
613,IO4JHIQS,how can someone stop it ?,Alcohol
614,1DS3P1XO,i feel unworthy,Depression


# Checking Word2Vec, Wordnet, EDA synonym replacement. 

In [6]:
# Download Word2Vec model from 
# https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

model = gensim.models.KeyedVectors.load_word2vec_format('../w2v_model/GoogleNews-vectors-negative300.bin.gz', binary=True)

w2v_aug = Word2vec(model = model,  runs=1)
w2v_aug.augment("i am stressed due to lack of financial support")

'i tcm_mon emphasized due to lacking of banking assistance'

In [50]:
wordnet_aug = Wordnet(v=False ,n=True, p=0.5)
wordnet_aug.augment('I am stressed due to lack of financial support')

'iodin am stressed due to lack of financial support'

In [61]:
EDA_aug = EDA()
EDA_aug.synonym_replacement("Is bhang good")

'Is bhang estimable'

# Using Synonym replacement, Random deletion, Random Swap in the EDA augmentation

In [ ]:
ID = []
orig_text = []
aug_EDA_synonym_replacement = []
aug_EDA_random_deletion = []
aug_EDA_random_swap = []
aug_EDA_random_insertion = []
label = []

for row, data in train_DF.iterrows():
    
    aug = EDA_aug.synonym_replacement(data['text'])
    aug_EDA_synonym_replacement.append(aug)
    
    aug = EDA_aug.random_deletion(data['text'])
    aug_EDA_random_deletion.append(aug)
    
    aug = EDA_aug.random_swap(data['text'])
    aug_EDA_random_swap.append(aug)
    
    aug = EDA_aug.random_insertion(data['text'])
    aug_EDA_random_insertion.append(aug)
    
    ID.append(data['ID'])
    orig_text.append(data['text'])
    label.append(data['label'])

In [86]:
sample = pd.DataFrame({'Orig_text': orig_text,
                       'aug_EDA_synonym_replacement': aug_EDA_synonym_replacement,
                       'aug_EDA_random_deletion': aug_EDA_random_deletion,
                       'aug_EDA_random_swap': aug_EDA_random_swap
                       #,'aug_EDA_random_insertion': aug_EDA_random_insertion
                      })
                       
sample

,Orig_text,aug_EDA_synonym_replacement,aug_EDA_random_deletion,aug_EDA_random_swap
0,i feel that it was better i die am happy,i feel that it was better i choke am happy,i feel that it was better die am happy,i better that it was feel i die am happy
1,why do i get hallucinations ?,why do i get hallucination ?,why do get hallucinations ?,why do i hallucinations get ?
2,i am stressed due to lack of financial support...,i am stressed due to lack of financial stomach...,i am stressed due to lack of financial support...,i am stressed school to lack of financial supp...
3,why is life important ?,why is aliveness important ?,why is life important ?,why is important life ?
4,how could i be helped to go through the depres...,how could i be helped to croak through the dep...,could i be to through depression ?,how could i be helped to go depression the thr...
...,...,...,...,...
611,what should i do to stop alcoholism ?,what should i do to stop inebriation ?,what should i do to stop alcoholism ?,should what i do to stop alcoholism ?
612,how to become my old self again,how to become my quondam self again,how to become my old self again,how to become my again self old
613,how can someone stop it ?,how can someone stopover it ?,how can someone stop it ?,how can someone ? it stop
614,i feel unworthy,i feel worthless,i feel,feel i unworthy


In [94]:
final_text = orig_text + aug_EDA_synonym_replacement + aug_EDA_random_deletion + aug_EDA_random_swap
len(final)
final_ID = ID*4
final_label = label*4

final_DF = pd.DataFrame({'ID' : final_ID,
                         'text' : final_text,
                         'label' :final_label})

final_DF.to_csv('../data/train_corrected_4.csv', index = False)

# Using synsets replacement in the wordnet

In [139]:
# Data augumentaion using wornet synonym method
def word_net_similarity(sentence):
    
    sent_split = sentence.split()
    aug_sent = [sentence]
    for word in sent_split:
        #print(word)
        if len(word)>4:
            for syn in wordnet.synsets(word): 
                for l in syn.lemmas(): 
                    #print("----", l.name())
                    temp_sent = sent_split.copy()
                    temp_sent[temp_sent.index(word)] = l.name().replace("_", " ").replace("-", " ")
                    #print(temp_sent)
                    aug_sent.append(" ".join(temp_sent))
    #print(aug_sent)                    
    #print("")
    return aug_sent

aug_final = []
label_final = []
final_ID = []
for row, data in train_DF.iterrows():
    aug = word_net_similarity(data['text'])
    label = [data['label']] * len(aug)
    ID = [data['ID']] * len(aug)
    aug_final.extend(aug)
    label_final.extend(label)
    final_ID.extend(ID)
len(aug_final), len(label_final)


aug_DF = pd.DataFrame({"ID" : final_ID,
                       "text":aug_final,
                       "label":label_final})

print(train_DF.shape)
print(aug_DF.shape)
aug_DF = aug_DF.drop_duplicates()
print(aug_DF.shape)

(616, 3)
(12031, 3)
(7939, 3)


In [140]:
aug_DF.to_csv('../data/train_corrected_4.csv', index = False)